<a href="https://colab.research.google.com/github/nhareesha/MLAI/blob/main/LLM/finetuned/Attempt_5_sql_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Need A100 processing units to run.

In [1]:
pip install pandas datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import os
import torch
import random
import numpy as np
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

# Step 0: Set Seed for Reproducibility

In [3]:
# Step 0: Set Seed for Reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)  # Set your seed here

# Step 1: Load the Spider Dataset


In [4]:
def load_spider_data():
    # Load the Spider dataset
    dataset = load_dataset('spider', split='train')  # Use the split you need (train, validation, test)
    return dataset

In [5]:
ds = load_spider_data()
type(ds)
print(ds[:2])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

{'db_id': ['department_management', 'department_management'], 'query': ['SELECT count(*) FROM head WHERE age  >  56', 'SELECT name ,  born_state ,  age FROM head ORDER BY age'], 'question': ['How many heads of the departments are older than 56 ?', 'List the name, born state and age of the heads of departments ordered by age.'], 'query_toks': [['SELECT', 'count', '(', '*', ')', 'FROM', 'head', 'WHERE', 'age', '>', '56'], ['SELECT', 'name', ',', 'born_state', ',', 'age', 'FROM', 'head', 'ORDER', 'BY', 'age']], 'query_toks_no_value': [['select', 'count', '(', '*', ')', 'from', 'head', 'where', 'age', '>', 'value'], ['select', 'name', ',', 'born_state', ',', 'age', 'from', 'head', 'order', 'by', 'age']], 'question_toks': [['How', 'many', 'heads', 'of', 'the', 'departments', 'are', 'older', 'than', '56', '?'], ['List', 'the', 'name', ',', 'born', 'state', 'and', 'age', 'of', 'the', 'heads', 'of', 'departments', 'ordered', 'by', 'age', '.']]}


# Step 2: Prepare the SQLDataset class


In [6]:
class SQLDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data[idx]['question']  # Get question
        answer = self.data[idx]['query']        # Get corresponding SQL from 'query' key

        # Tokenize the inputs and labels
        inputs = self.tokenizer(
            question, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        # You can choose which tokens to use for labels:
        # Here we can use both the actual SQL query and the tokenized versions
        labels = self.tokenizer(
            answer, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        # Tokenize `query_toks` and `query_toks_no_value` if needed
        query_toks = self.data[idx]['query_toks']
        query_toks_no_value = self.data[idx]['query_toks_no_value']

        # You might want to process query_toks and query_toks_no_value for additional insights or features

        # Flatten the tensor dimensions
        input_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()
        labels = labels.input_ids.squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'query_toks': query_toks,  # Return raw tokenized query
            'query_toks_no_value': query_toks_no_value  # Return raw tokenized query without values
        }



# Step 3: Load Pre-Trained Model and Tokenizer


In [7]:
model_name = 't5-small'  # or the name of your trained model
tokenizer = T5Tokenizer.from_pretrained(model_name)
original_model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
# Model for fine tuning
model_name = 't5-small'  # or the name of your trained model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#TEMPORARY

In [ ]:
# Load the model and tokenizer from the checkpoint directory
# Set your output directory and checkpoint directory
checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/checkpoint-100'

model = T5ForConditionalGeneration.from_pretrained(checkpoint_dir)
tokenizer = T5Tokenizer.from_pretrained(checkpoint_dir)

OSError: Can't load tokenizer for '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/checkpoint-100'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/checkpoint-100' is the correct path to a directory containing all relevant files for a T5Tokenizer tokenizer.

# Load previous checkpoint if exists

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Move the model to GPU if available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [22]:
# Create output directory and checkpoint directory if they don't exist
output_dir = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5'
checkpoint_dir = os.path.join(output_dir, 'checkpoint')
os.makedirs(checkpoint_dir, exist_ok=True)


#IGNORE for sometime

In [11]:
# Set your output directory and checkpoint directory
output_dir = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5'

checkpoint_path = f'{output_dir}/checkpoint/Aug23'

In [12]:
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
else:
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
    start_epoch = 0


# Step 4: Load and Prepare Data



In [23]:
start_index = 1
end_index = 1000

In [24]:

# Load the Spider Dataset
spider_dataset_everything = load_spider_data()

# Select based in indexes
spider_dataset = spider_dataset_everything.select(range(start_index, end_index))

# Define the sizes for the splits
train_size = int(0.8 * len(spider_dataset))
val_size = int(0.1 * len(spider_dataset))  # 10% for validation
test_size = len(spider_dataset) - train_size - val_size  # Remaining 10% for testing

# Split the dataset into train, validation, and test sets
train_dataset, temp_dataset = torch.utils.data.random_split(spider_dataset, [train_size, len(spider_dataset) - train_size])
val_dataset, test_dataset = torch.utils.data.random_split(temp_dataset, [val_size, test_size])

# Now, train_dataset, val_dataset, and test_dataset are ready to use
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")


Training set size: 799
Validation set size: 99
Test set size: 101


# Create DataLoaders

In [25]:
# Create DataLoaders
train_data_loader = DataLoader(SQLDataset(train_dataset, tokenizer), batch_size=8, shuffle=True)
val_data_loader = DataLoader(SQLDataset(val_dataset, tokenizer), batch_size=8)
test_data_loader = DataLoader(SQLDataset(test_dataset, tokenizer), batch_size=8)


# Step 5: Setup Training Arguments

In [26]:
training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy='epoch',
    learning_rate=5e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    save_total_limit=2,  # Limit the total amount of checkpoints, delete the older checkpoints.
    save_steps=25,      # Save checkpoint every 500 steps

)

# Step 6: Initialize Trainer

In [27]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_loader.dataset,
    eval_dataset=val_data_loader.dataset,
)

# Step 7: Train the Model

#### Train the Model with Checkpointing


In [29]:
for epoch in range(start_epoch, training_args.num_train_epochs):
    trainer.train()

    # Save the model checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint-{epoch + 1}.pt')  # Specify the file name

    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)


Epoch,Training Loss,Validation Loss
1,No log,0.027300
2,No log,0.022357
3,No log,0.021817


Epoch,Training Loss,Validation Loss
1,No log,0.020360
2,No log,0.018250
3,No log,0.017890


Epoch,Training Loss,Validation Loss
1,No log,0.018708
2,No log,0.016192
3,No log,0.015209


# Step 8: Evaluate the Model


In [30]:
# Step 8: Evaluate the Model
results = trainer.evaluate()

In [31]:
# Print evaluation results
print(f"Evaluation results: {results}")

Evaluation results: {'eval_loss': 0.015208528377115726, 'eval_runtime': 0.7871, 'eval_samples_per_second': 125.779, 'eval_steps_per_second': 16.516, 'epoch': 3.0}


# Step 9: Generate Predictions on Test Set -------- NEED TO FIX

In [32]:
predictions = []
actuals = []

for batch in test_data_loader:
    input_ids = batch['input_ids'].to(model.device)
    attention_mask = batch['attention_mask'].to(model.device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_sql = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    actual_sql = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

    predictions.extend(predicted_sql)
    actuals.extend(actual_sql)


RuntimeError: each element in list of batch should be of equal size

# Compare predicted SQL with actual SQL

In [ ]:
# Compare predicted SQL with actual SQL
for pred, actual in zip(predictions, actuals):
    print(f"Predicted SQL: {pred}\nActual SQL: {actual}\n")

# Step 10: Save the Fine-Tuned Model and Tokenizer

In [34]:
# Define the output directory
output_dir = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save the fine-tuned model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model/added_tokens.json')

In [33]:
# Step 10: Save the Fine-Tuned Model and Tokenizer
model.save_pretrained('./fine_tuned_attempt_t5')
tokenizer.save_pretrained('./fine_tuned_attempt_t5_tokenizer')

('./fine_tuned_attempt_t5_tokenizer/tokenizer_config.json',
 './fine_tuned_attempt_t5_tokenizer/special_tokens_map.json',
 './fine_tuned_attempt_t5_tokenizer/spiece.model',
 './fine_tuned_attempt_t5_tokenizer/added_tokens.json')

# Step 11: Using the Model for Prompting

In [39]:
def generate_sql(question, max_length):
    input_ids = tokenizer.encode(question, return_tensors='pt').to(model.device)
    with torch.no_grad():
        output = model.generate(input_ids, max_new_tokens=max_length)
    generated_sql = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_sql


In [36]:
# Example prompt usage
question = "How many heads of the departments are older than 56?"
predicted_sql_query = generate_sql(question)
print(f"Generated SQL Query: {predicted_sql_query}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated SQL Query: SELECT count(*) FROM department WHERE age > 56


# Smaller dataset results in underfitting - HENCE model is not able to learn much.

Things to do

1. Increase training set
2. Adjust learning rate
3. Create tailored prompts.

In [41]:
question='For those employees who did not have any job in the past, give me the comparison about the amount of job_id over the job_id , and group by attribute job_id, and list from low to high by the JOB_ID please.'

predicted_sql_query = generate_sql(question, 200)
print(f"Generated SQL Query: {predicted_sql_query}")

Generated SQL Query: SELECT count(*) FROM Employee WHERE job_id NOT IN (SELECT eid FROM Employee) GROUP BY eid ORDER BY count(*) DESC LIMIT 1


In [46]:
question='Create an SQL INSERT statement for the Employee table.'

predicted_sql_query = generate_sql(question, 2000)
print(f"Generated SQL Query: {predicted_sql_query}")

Generated SQL Query: SELECT DISTINCT T1.SupportRepId FROM Employee AS T1 JOIN Statement AS T2 ON T1.SupportRepId = T2.EmployeeId WHERE T1.Table_name = "SupportRepId"
